In [211]:
import string
import pandas as pd

In [212]:
train_file = './data/sr_set-ud-train.conllu'
test_file = './data/sr_set-ud-test.conllu'
dev_file = './data/sr_set-ud-dev.conllu'

In [213]:
lines = []
with open(train_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()
with open(test_file, 'r', encoding='utf-8') as f:
    lines.extend(f.readlines())
with open(dev_file, 'r', encoding='utf-8') as f:
    lines.extend(f.readlines())

In [214]:
open_class_tags = ['ADJ', 'ADV', 'INTJ', 'NOUN', 'PROPN', 'VERB']

In [215]:
ud_content = []
for line in lines:
    if len(line) == 1:
        continue
    if line.startswith("#"):
        continue
    
    parts = line.split('\t') 
    word = parts[1]
    pos_tag = parts[3]
    if pos_tag not in open_class_tags:
        continue
    ud_content.append(word)

In [216]:
len(ud_content)

54632

# Preprocessing

All words are converted to lowercase.

In [217]:
ud_content = [word.lower() for word in ud_content]

Everything that contains digits and punctuation is removed.

In [218]:
ud_content = [word for word in ud_content if all(not c.isdigit() and c not in string.punctuation for c in word)]

In [219]:
data = pd.Series(ud_content)
data_freqs = data.value_counts()
data_freqs[:20]

rekao         332
godine        279
eu            261
takođe        166
bih           151
kada          141
kaže          132
zemlje        131
kosova        128
međutim       126
samo          120
evra          117
još           117
srbije        110
izjavio       107
trebalo       107
predsednik    106
odsto          99
više           98
ministar       94
dtype: int64

# Lexicon structure

Lexicon will be represented as a structure which consists of lemma (the base form), its tag and a list of the derived forms.
Each derived form consits of the form itself and its tag.

In [220]:
class LexiconEntry:
    def __init__(self, lemma, tag):
        self.lemma = lemma
        self.tag = tag
        self.derived_forms = []

In [221]:
lexicon = []

Defining useful functions:

In [222]:
def print_lexicon():
    for lexicon_entry in lexicon:
        lemma = lexicon_entry.lemma
        tag = lexicon_entry.tag

        print("{} [{}]".format(lemma, tag))
        if len(lexicon_entry.derived_forms) == 0:
            continue
        for idx, derived_form_entry in enumerate(lexicon_entry.derived_forms):
            derived_form, derived_form_info = derived_form_entry
            
            if idx == 0:
                print("    ∟ {} {}".format(derived_form, derived_form_info))
            else:
                print("      {} {}".format(derived_form, derived_form_info))

In [223]:
def add_to_lexicon(lemma, base_tag, derived_forms = []):
    global ud_content
    lexicon_entry = LexiconEntry(lemma, base_tag)
    for lex in lexicon:
        if lex.lemma == lemma and lex.tag == base_tag:
            return
    
    ud_content = [word for word in ud_content if word != lemma]

    for der_form in derived_forms:
        lexicon_entry.derived_forms.append(der_form)
    lexicon.append(lexicon_entry)
    
    derived_forms_words = [w[0] for w in derived_forms]
    ud_content = [word for word in ud_content if word not in derived_forms_words]

In [224]:
def get_lexicon_entry(lemma, tag):
    for lex_entry in lexicon:
        if lex_entry.lemma == lemma and lex_entry.tag == tag:
            return lex_entry
        
    return None

Let's start with the most distinctive suffixes, which can be easily be observed

# 1. Verbs

## 1.1 Infinitive

All verbs in Serbian end with: *-ti* or *-ći*

In [225]:
infinitive_verbs_ITI = list(set([w for w in ud_content if w.endswith('iti')]))

In [226]:
infinitive_verbs_ITI

['izmeniti',
 'uskladiti',
 'napraviti',
 'prenositi',
 'pridružiti',
 'posvetiti',
 'rušiti',
 'snositi',
 'nuditi',
 'odobriti',
 'opslužiti',
 'razotkriti',
 'oprostiti',
 'predložiti',
 'izgubiti',
 'odgovoriti',
 'zaključiti',
 'otvoriti',
 'suočiti',
 'težiti',
 'rijaliti',
 'izvršiti',
 'omogućiti',
 'pružiti',
 'naslediti',
 'razvodniti',
 'uhvatiti',
 'sprovoditi',
 'voditi',
 'ograničiti',
 'posetiti',
 'slediti',
 'protiviti',
 'odrediti',
 'uspostaviti',
 'stvoriti',
 'smanjiti',
 'otkloniti',
 'ostvariti',
 'protumačiti',
 'završiti',
 'napustiti',
 'izgraditi',
 'preporučiti',
 'zameniti',
 'nastupiti',
 'uložiti',
 'pratiti',
 'namestiti',
 'odvratiti',
 'učiniti',
 'povrediti',
 'nadoknaditi',
 'povratiti',
 'unaprediti',
 'osetiti',
 'ustanoviti',
 'razdvojiti',
 'umoriti',
 'usuditi',
 'arvaniti',
 'poslužiti',
 'primiti',
 'zavisiti',
 'predstaviti',
 'izraditi',
 'kupiti',
 'ponuditi',
 'usredsrediti',
 'ostaviti',
 'različiti',
 'ponoviti',
 'potvrditi',
 'uslediti

In [227]:
for inf_verb in infinitive_verbs_ITI:
    add_to_lexicon(inf_verb, 'VERB')

In [228]:
len(ud_content)

53283

In [229]:
infinitive_verbs_ETI = list(set([w for w in ud_content if w.endswith('eti')]))

In [230]:
infinitive_verbs_ETI

['poseti',
 'aveti',
 'dopreti',
 'razumeti',
 'meti',
 'posveti',
 'preti',
 'preduzeti',
 'preneti',
 'preoteti',
 'peti',
 'oteti',
 'univerziteti',
 'ćančeti',
 'našteti',
 'početi',
 'podneti',
 'dijeti',
 'videti',
 'animoziteti',
 'oživeti',
 'tereti',
 'uspeti',
 'doprineti',
 'započeti',
 'predvideti',
 'anketi',
 'preuzeti',
 'doneti',
 'predmeti',
 'antikviteti',
 'deveti',
 'oseti',
 'fakulteti',
 'ahmeti',
 'uzeti']

In [231]:
for inf_verb in infinitive_verbs_ETI:
    add_to_lexicon(inf_verb, 'VERB')

In [232]:
infinitive_verbs_ATI = list(set([w for w in ud_content if w.endswith('ati')]))

In [233]:
infinitive_verbs_ATI

['sećati',
 'postati',
 'menjati',
 'imati',
 'sindikati',
 'izabrati',
 'palati',
 'očekivati',
 'falsifikovati',
 'zloupotrebljavati',
 'vraćati',
 'rezultati',
 'pokušavati',
 'ostati',
 'investirati',
 'zamišljati',
 'dočekivati',
 'uhvati',
 'artikulisati',
 'garantovati',
 'otputovati',
 'sastati',
 'isplati',
 'aktivirati',
 'sarađivati',
 'postupati',
 'okončati',
 'odvijati',
 'trajati',
 'organizovati',
 'stajati',
 'zapitati',
 'pružati',
 'kopirati',
 'plaćati',
 'zahtevati',
 'glasati',
 'cenjkati',
 'prikazati',
 'potpisati',
 'poboljšati',
 'ratifikovati',
 'sačuvati',
 'birati',
 'poznati',
 'povećati',
 'putovati',
 'blokirati',
 'morati',
 'posmatrati',
 'primenjivati',
 'igrati',
 'pozvati',
 'predati',
 'obavljati',
 'oklevati',
 'klati',
 'predstavljati',
 'raspravljati',
 'preduzimati',
 'postojati',
 'ometati',
 'fokusirati',
 'prebacivati',
 'zadržati',
 'preispitati',
 'prisustvovati',
 'intervenisati',
 'održati',
 'doputovati',
 'hrvati',
 'reagovati',
 'sati

In [234]:
for inf_verb in infinitive_verbs_ATI:
    add_to_lexicon(inf_verb, 'VERB')

### 1.2 Past participle

Ending with: *-vši*

In [235]:
past_participle_VSI = list(set([w for w in ud_content if w.endswith('vši')]))
print(past_participle_VSI)

['bivši', 'ponovivši', 'postavivši', 'dobivši', 'pobedivši', 'zamenivši', 'rekavši', 'naslutivši', 'učinivši', 'primetivši', 'osvojivši', 'pohvalivši', 'izazvavši', 'pretvorivši', 'počevši', 'proširivši', 'uzdrmavši', 'dostigavši', 'zabeleživši', 'završivši']


In [236]:
for pp_VSI in past_participle_VSI:
    add_to_lexicon(pp_VSI, 'VERB')

### 1.3 Present participle

Ending with: *-jući*

In [237]:
def add_to_baseform(derived_form, derived_form_info, tag, src_replace, dst_replace):
    for lexicon_entry in lexicon:
        if lexicon_entry.lemma == derived_form.replace(src_replace, dst_replace) and lexicon_entry.tag == tag:
            lexicon_entry.derived_forms.append((derived_form, derived_form_info))
            return
    
    lemma = derived_form.replace(src_replace, dst_replace)
    print("Opsasa {} -> {}".format(derived_form, lemma))
    derived_forms = [(derived_form, derived_form_info)]
    add_to_lexicon(lemma, tag, derived_forms)

In [238]:
present_participle_JUCI = list(set([w for w in ud_content if w.endswith('jući')]))

for pp in present_participle_JUCI:
    if pp.endswith('ajući'):
        add_to_baseform(pp, "PRESENT_PAT", "VERB", "ajući", "ati")

    elif pp.endswith('ujući'):
        add_to_baseform(pp, "PRESENT_PAT", "VERB", "ujući", "ovati")

    else:
        print("Problem: " + pp)

Opsasa obećavajući -> obećavati
Opsasa preneražavajući -> preneražavati
Opsasa pretvarajući -> pretvarati
Opsasa uveravajući -> uveravati
Opsasa opstruirajući -> opstruirati
Opsasa emigrirajući -> emigrirati
Opsasa otvarajući -> otvarati
Opsasa obeležavajući -> obeležavati
Opsasa bacajući -> bacati
Opsasa ukazujući -> ukazovati
Opsasa otuđujući -> otuđovati
Opsasa naglašavajući -> naglašavati
Opsasa prodajući -> prodati
Opsasa zbližavajući -> zbližavati
Opsasa ubacujući -> ubacovati
Opsasa tragajući -> tragati
Opsasa nedostajući -> nedostati
Opsasa nazivajući -> nazivati
Opsasa favorizujući -> favorizovati
Opsasa savetujući -> savetovati
Opsasa ponavljajući -> ponavljati
Opsasa najavljujući -> najavljovati
Opsasa dodajući -> dodati
Opsasa odgovarajući -> odgovarati
Opsasa čitajući -> čitati
Opsasa primajući -> primati
Opsasa svrstavajući -> svrstavati
Opsasa zatvarajući -> zatvarati
Opsasa saopštavajući -> saopštavati
Opsasa znajući -> znati
Opsasa optužujući -> optužovati
Opsasa primo

In [239]:
present_participle_JUCI

['obećavajući',
 'preneražavajući',
 'insistirajući',
 'pretvarajući',
 'uveravajući',
 'izražavajući',
 'opstruirajući',
 'emigrirajući',
 'otvarajući',
 'obraćajući',
 'obeležavajući',
 'bacajući',
 'ukazujući',
 'otuđujući',
 'naglašavajući',
 'prodajući',
 'zbližavajući',
 'ubacujući',
 'tragajući',
 'nedostajući',
 'nazivajući',
 'favorizujući',
 'savetujući',
 'ponavljajući',
 'najavljujući',
 'dodajući',
 'odgovarajući',
 'postavljajući',
 'čitajući',
 'primajući',
 'obavljajući',
 'svrstavajući',
 'zatvarajući',
 'saopštavajući',
 'znajući',
 'optužujući',
 'primoravajući',
 'oslobađajući',
 'narastajući',
 'izdvajajući',
 'pozdravljajući',
 'igrajući',
 'odbijajući',
 'predstavljajući',
 'uključujući',
 'potiskujući',
 'objašnjavajući',
 'predsedavajući',
 'čekajući',
 'završavajući',
 'potvrđujući',
 'ohrabrujući',
 'dajući',
 'preduzimajući',
 'prezajući',
 'sprečavajući',
 'diskreditujući',
 'protestujući',
 'suočavajući',
 'omogućavajući',
 'upozoravajući',
 'pozivajući',


Another type of suffix of present participle form is *-eći*, but it can be mixed with infinitive form.

In [240]:
suffix_ECI = list(set([w for w in ud_content if w.endswith('eći')]))

In [241]:
suffix_ECI

['govoreći',
 'misleći',
 'preći',
 'baveći',
 'navodeći',
 'poreći',
 'deleći',
 'doprinoseći',
 'odlazeći',
 'najveći',
 'trudeći',
 'donoseći',
 'kršeći',
 'tvrdeći',
 'postojeći',
 'treći',
 'koristeći',
 'preseći',
 'sledeći',
 'reći',
 'paleći',
 'steći',
 'nesreći',
 'sudeći',
 'izlazeći',
 'predstojeći',
 'važeći',
 'tražeći',
 'beležeći',
 'veći']

# 2. Adjectives

## 2.1 Comparative

Suffixes:
- Singular: *-niji (m.), -nija (f.), -nije (n.)*
- Plural: *-nijim

In [242]:
comparative_forms = [w for w in ud_content if w.endswith('niji') or w.endswith('nija') or w.endswith('nije') or w.endswith('nijim')]
comparative_forms = [w for w in comparative_forms if not w.startswith('naj')]
comparative_forms = list(set(comparative_forms))

In [243]:
comparative_forms

['menija',
 'decenije',
 'ceremonije',
 'rumuniji',
 'unija',
 'estonije',
 'demokratičnija',
 'sigurnija',
 'linije',
 'španija',
 'makedonija',
 'konkurentnija',
 'zainteresovanija',
 'čečeniji',
 'letonije',
 'alianija',
 'španiji',
 'svestranije',
 'unije',
 'urgentniji',
 'kompaniji',
 'beskompromisniji',
 'britanija',
 'ranije',
 'optimalnije',
 'siromašnije',
 'potpunije',
 'kompanija',
 'kompleksnija',
 'važnija',
 'ceremonija',
 'kasnije',
 'španije',
 'slovenija',
 'striktnije',
 'precizniji',
 'optimističniji',
 'rasprostranjenija',
 'jevtinijim',
 'rumunije',
 'ranijim',
 'britaniji',
 'važnije',
 'bezbedniji',
 'telefonije',
 'apolonija',
 'transparentnijim',
 'uticajniji',
 'slovenije',
 'sigurnije',
 'slobodniji',
 'makedonije',
 'pensilvaniji',
 'jeftinije',
 'liniji',
 'decenija',
 'efikasnija',
 'značajniji',
 'pozitivniji',
 'snažnije',
 'albanija',
 'britanije',
 'ozbiljniji',
 'albanije',
 'sloveniji',
 'značajnije',
 'albaniji',
 'rasprostranjeniji',
 'uniji',
 'k

We will build positive forms of the comparative forms:

-niji $\rightarrow$ -an

-nija $\rightarrow$ -na

-nije $\rightarrow$ -no

-nijim $\rightarrow$ -an 
    - there's no way of knowing if the comparative form is supposed to be in masculine, feminine or neutral form, so here it is deliberately chosen that the masculine form will be the base form

In [244]:
for comparative in comparative_forms:
    positive_lexicon_form = None
    if comparative.endswith("niji"):
        positive = comparative.replace("niji", "an")
        positive_lexicon_form = (comparative, 'COMP-M-SG')
    if comparative.endswith("nija"):
        positive = comparative.replace("nija", "na")
        positive_lexicon_form = (comparative, 'COMP-F-SG')
    if comparative.endswith("nije"):
        positive = comparative.replace("nije", "no")
        positive_lexicon_form = (comparative, 'COMP-N-SG')
    if comparative.endswith("nijim"):
        positive = comparative.replace("nijim", "an")
        positive_lexicon_form = (comparative, 'COMP-M-PL')
    
    add_to_lexicon(positive, 'ADJ', [positive_lexicon_form])

In [245]:
print_lexicon()

izmeniti [VERB]
uskladiti [VERB]
napraviti [VERB]
prenositi [VERB]
pridružiti [VERB]
posvetiti [VERB]
rušiti [VERB]
snositi [VERB]
nuditi [VERB]
odobriti [VERB]
opslužiti [VERB]
razotkriti [VERB]
oprostiti [VERB]
predložiti [VERB]
izgubiti [VERB]
odgovoriti [VERB]
zaključiti [VERB]
otvoriti [VERB]
suočiti [VERB]
težiti [VERB]
rijaliti [VERB]
izvršiti [VERB]
omogućiti [VERB]
pružiti [VERB]
naslediti [VERB]
razvodniti [VERB]
uhvatiti [VERB]
sprovoditi [VERB]
voditi [VERB]
ograničiti [VERB]
posetiti [VERB]
slediti [VERB]
protiviti [VERB]
odrediti [VERB]
uspostaviti [VERB]
stvoriti [VERB]
smanjiti [VERB]
otkloniti [VERB]
ostvariti [VERB]
protumačiti [VERB]
završiti [VERB]
napustiti [VERB]
izgraditi [VERB]
preporučiti [VERB]
zameniti [VERB]
nastupiti [VERB]
uložiti [VERB]
pratiti [VERB]
namestiti [VERB]
odvratiti [VERB]
učiniti [VERB]
povrediti [VERB]
nadoknaditi [VERB]
povratiti [VERB]
unaprediti [VERB]
osetiti [VERB]
ustanoviti [VERB]
razdvojiti [VERB]
umoriti [VERB]
usuditi [VERB]
arvani

## 2.2 Superlative

It is fairly easy to find adjectives in superlative form because they almost always have the prefix *naj-*.

In [246]:
prefix_NAJ = list(set([w for w in ud_content if w.startswith('naj')]))

In [247]:
prefix_NAJ

['najvažnije',
 'najizolovanijih',
 'najbezbednijim',
 'najsličnija',
 'najčešće',
 'najkraća',
 'najmoćnije',
 'najveći',
 'najnovije',
 'najveću',
 'najuglednijih',
 'najčuvenija',
 'najvišeg',
 'najvišim',
 'najvažnijim',
 'najavljena',
 'najozbiljniji',
 'najjednostavniji',
 'najvećeg',
 'najavljen',
 'najtiražnijeg',
 'najblaže',
 'najjača',
 'najpouzdniji',
 'najgore',
 'najboljih',
 'najozbiljnija',
 'najjače',
 'najvažnijih',
 'najvažnija',
 'najboljim',
 'najpopularnija',
 'najdelikatnijoj',
 'najugledniji',
 'najprodavanije',
 'najniži',
 'najvažniju',
 'najbezbednijih',
 'najnoviju',
 'najvećih',
 'najstarijim',
 'najneslobodnija',
 'najznačajnije',
 'najbolja',
 'najavljuje',
 'najdinamičnija',
 'najčuvanijih',
 'najurgentniji',
 'najavljenom',
 'najmanju',
 'najavile',
 'najverovatnije',
 'najtraženija',
 'najslobodnija',
 'najpoznatijih',
 'najesen',
 'najverovatniji',
 'najnovijim',
 'najzad',
 'najisplativija',
 'najteže',
 'najnovija',
 'najnovijih',
 'najmoćniji',
 'n

In [248]:
def is_superlative_of_existing_base_form(lexicon_entry, superlative):
    if len(lexicon_entry.derived_forms) == 0:
        return False
    
    comparative = superlative[3:]
    for d_form in lexicon_entry.derived_forms:
        if d_form[1].startswith("COMP-") and d_form[0].startswith(comparative):
            gender = d_form[1].split("-")[1]
            number = d_form[1].split("-")[2]
            lexicon_entry.derived_forms.append((superlative, "SUP-{}-{}".format(gender, number)))
            return True
        
    return False
            

for p_NAJ in prefix_NAJ:
    lexicon_entries = [l for l in lexicon if is_superlative_of_existing_base_form(l, p_NAJ)]

In [249]:
print_lexicon()

izmeniti [VERB]
uskladiti [VERB]
napraviti [VERB]
prenositi [VERB]
pridružiti [VERB]
posvetiti [VERB]
rušiti [VERB]
snositi [VERB]
nuditi [VERB]
odobriti [VERB]
opslužiti [VERB]
razotkriti [VERB]
oprostiti [VERB]
predložiti [VERB]
izgubiti [VERB]
odgovoriti [VERB]
zaključiti [VERB]
otvoriti [VERB]
suočiti [VERB]
težiti [VERB]
rijaliti [VERB]
izvršiti [VERB]
omogućiti [VERB]
pružiti [VERB]
naslediti [VERB]
razvodniti [VERB]
uhvatiti [VERB]
sprovoditi [VERB]
voditi [VERB]
ograničiti [VERB]
posetiti [VERB]
slediti [VERB]
protiviti [VERB]
odrediti [VERB]
uspostaviti [VERB]
stvoriti [VERB]
smanjiti [VERB]
otkloniti [VERB]
ostvariti [VERB]
protumačiti [VERB]
završiti [VERB]
napustiti [VERB]
izgraditi [VERB]
preporučiti [VERB]
zameniti [VERB]
nastupiti [VERB]
uložiti [VERB]
pratiti [VERB]
namestiti [VERB]
odvratiti [VERB]
učiniti [VERB]
povrediti [VERB]
nadoknaditi [VERB]
povratiti [VERB]
unaprediti [VERB]
osetiti [VERB]
ustanoviti [VERB]
razdvojiti [VERB]
umoriti [VERB]
usuditi [VERB]
arvani

- Prisvojni pridevi (mesta): -ski, -čki, -ški
- Prisvojni pridevi (?): -ni
- Prisvojni pridevi (ljudi): -ov, -in, -ev

## 2.3 Possessive adjectives

Possessive adjectives can be of the following forms depending on the type of entity that *possesses*:

1. *-ov, -ev, -in*: when the entity that possesses is a human being
    - Examples: Markov "Mark's", Tonijev "Tony's", Nemanjin "Nemanja's" 
2. *-ski, -čki, -ški*: usually with toponymical entities - countries, ethnicities, cities, villages, regions, etc.
    - Examples: srpski "Serbian", nemački "German", praški "Prague's"
3. *-ni*
    - Examples: državni "state"

Of course, adjectives are changed according to gender as well, so we have:

2. *-ska* (f.), *-sko* (n.), *-čka* (f.), *-čko (n.), *-ška* (f.), *-ško* (n.)
3. *-na* (f.), *-no* (n.)

However, the inflection forms for masculine, feminine and neutral gender are the same when inflecting over different cases. The only difference is in the nominative form, so feminine and neutral forms are put under the masculine form.

Singular

| case | masculine | feminine | neutral | 
|------|-----------|----------|---------|
|nominative|srpski|srpska|srpsko|
|genitive|srpskog|srpske|srpskog|
|dative|srpskom|srpskoj|srpskom|
|acusative|srpski|srpsku|srpsko|
|vocative|srpski|srpska|srpsko|
|instrumental|srpskim|srpskom|srpskim|
|locative|srpskom|srpskoj|srpskom|

Plural

| case | masculine | feminine | neutral | 
|------|-----------|----------|---------|
|nominative|srpski|srpske|srpska|
|genitive|srpskih|srpskih|srpske|
|dative|srpskim|srpskim|srpskoj|
|acusative|srpske|srpske|srpsku|
|vocative|srpski|srpske|srpska|
|instrumental|srpskim|srpskim|srpskom|
|locative|srpskim|srpskim|srpskoj|

Possible siffixes: *-ski*, *-skog*, *-skom*, *-skim*, *-ska*, *-ske*, *-skoj*, *-sku*, *-skih*, *-sko*

### 2.3.1 suffix *-ski* and its inflections

In [250]:
possessive_adjectives = list(set([w for w in ud_content if w.endswith('ski')]))

In [251]:
possessive_adjectives

['istinski',
 'levičarski',
 'kiparski',
 'crnogorski',
 'španski',
 'mađarski',
 'birokratski',
 'ivanovski',
 'ekonomski',
 'manevski',
 'herkulovski',
 'detektivski',
 'briselski',
 'turski',
 'bliski',
 'irski',
 'lavovski',
 'avionski',
 'pakistanski',
 'solunski',
 'bugarski',
 'antonovski',
 'filmski',
 'engleski',
 'televizijski',
 'automatski',
 'akademski',
 'izraelski',
 'sofijski',
 'drumski',
 'azilski',
 'pariski',
 'argentinski',
 'budžetski',
 'vijetnamski',
 'medijski',
 'jugoslovenski',
 'milošoski',
 'vrhunski',
 'mančevski',
 'peševski',
 'sovjetski',
 'univerzitetski',
 'peruanski',
 'ljudski',
 'belgijski',
 'energetski',
 'svetski',
 'niski',
 'geografski',
 'kosovski',
 'većinski',
 'evropski',
 'verski',
 'vremenski',
 'proruski',
 'kurdski',
 'socijaldemokratski',
 'katastarski',
 'ruski',
 'bankarski',
 'amaterski',
 'lisabonski',
 'bosforski',
 'švedski',
 'gruevski',
 'orvelovski',
 'holivudski',
 'terapeutski',
 'nikolovski',
 'francuski',
 'romski',
 'spo

In [252]:
for possesive_adj in possessive_adjectives:
    add_to_lexicon(possesive_adj, "ADJ")

In [253]:
def add_possessive_adjectives(possessive_adjectives, replace_src, replace_dst):
    for possessive_ADJ in possessive_adjectives:
        derived_f = (possessive_ADJ, "POSSESSIVE")
        base_form = possessive_ADJ.replace(replace_src, replace_dst)
        lexicon_entry = get_lexicon_entry(base_form, "ADJ")
        if lexicon_entry is not None:
            lexicon_entry.derived_forms.append(derived_f)
        else:
            add_to_lexicon(base_form, "ADJ", [derived_f])

In [254]:
def add_adjective_inflection_forms(inflection_suffixes):
    for suffix, base_suffix in inflection_suffixes:
        possessive_adjectives = list(set([w for w in ud_content if w.endswith(suffix)]))
        add_possessive_adjectives(possessive_adjectives, suffix, base_suffix)

In [255]:
inflection_suffixes = [('skog', 'ski'), ('skom', 'ski'), ('skim', 'ski'), ('ska', 'ski'), ('ske', 'ski'), 
                       ('skoj', 'ski'), ('sku', 'ski'), ('skih', 'ski'), ('sko', 'ski')]
add_adjective_inflection_forms(inflection_suffixes)

In [256]:
print_lexicon()

izmeniti [VERB]
uskladiti [VERB]
napraviti [VERB]
prenositi [VERB]
pridružiti [VERB]
posvetiti [VERB]
rušiti [VERB]
snositi [VERB]
nuditi [VERB]
odobriti [VERB]
opslužiti [VERB]
razotkriti [VERB]
oprostiti [VERB]
predložiti [VERB]
izgubiti [VERB]
odgovoriti [VERB]
zaključiti [VERB]
otvoriti [VERB]
suočiti [VERB]
težiti [VERB]
rijaliti [VERB]
izvršiti [VERB]
omogućiti [VERB]
pružiti [VERB]
naslediti [VERB]
razvodniti [VERB]
uhvatiti [VERB]
sprovoditi [VERB]
voditi [VERB]
ograničiti [VERB]
posetiti [VERB]
slediti [VERB]
protiviti [VERB]
odrediti [VERB]
uspostaviti [VERB]
stvoriti [VERB]
smanjiti [VERB]
otkloniti [VERB]
ostvariti [VERB]
protumačiti [VERB]
završiti [VERB]
napustiti [VERB]
izgraditi [VERB]
preporučiti [VERB]
zameniti [VERB]
nastupiti [VERB]
uložiti [VERB]
pratiti [VERB]
namestiti [VERB]
odvratiti [VERB]
učiniti [VERB]
povrediti [VERB]
nadoknaditi [VERB]
povratiti [VERB]
unaprediti [VERB]
osetiti [VERB]
ustanoviti [VERB]
razdvojiti [VERB]
umoriti [VERB]
usuditi [VERB]
arvani

### 2.3.2 suffix *-čki* and its inflections


Singular

| case | masculine | feminine | neutral | 
|------|-----------|----------|---------|
|nominative|nemački|nemačka|nemačko|
|genitive|nemačkog|nemačke|nemačkog|
|dative|nemačkom|nemačkoj|nemačkom|
|acusative|nemački|nemačku|nemačko|
|vocative|nemački|nemačka|nemačko|
|instrumental|nemačkim|nemačkom|nemačkim|
|locative|nemačkom|nemačkoj|nemačkom|

Plural

| case | masculine | feminine | neutral | 
|------|-----------|----------|---------|
|nominative|nemački|nemačke|nemačka|
|genitive|nemačkih|nemačkih|nemačke|
|dative|nemačkim|nemačkim|nemačkoj|
|acusative|nemačke|nemačke|nemačku|
|vocative|nemački|nemačke|nemačka|
|instrumental|nemačkim|nemačkim|nemačkom|
|locative|nemačkim|nemačkim|nemačkoj|

Possible suffixes: *-čki*, *-čkog*, *-čkom*, *-čka*, *-čkih*, *-čkim*, *-čko*, *-čke*, *-čkoj*, *-čku*

In [257]:
possessive_adjectives = list(set([w for w in ud_content if w.endswith('čki')]))

In [258]:
possessive_adjectives

['bošnjački',
 'američki',
 'muzički',
 'predsednički',
 'turistički',
 'etnički',
 'slovenački',
 'posmatrački',
 'železnički',
 'statistički',
 'putnički',
 'irački',
 'ultranacionalistički',
 'tehnički',
 'preduzetnički',
 'pacifički',
 'radnički',
 'višestranački',
 'teroristički',
 'ekstremistički',
 'pregovarački',
 'politički',
 'zajednički',
 'brodovlasnički',
 'stranački',
 'grčki',
 'prostački',
 'populistički',
 'nemački',
 'komunistički',
 'monopolistički',
 'umetnički',
 'međuetnički']

In [259]:
for possesive_adj in possessive_adjectives:
    add_to_lexicon(possesive_adj, "ADJ")

In [260]:
inflection_suffixes = [('čkog', 'čki'), ('čkom', 'čki'), ('čka', 'čki'), ('čkih', 'čki'), ('čkim', 'čki'), ('čko', 'čki'), 
                      ('čke', 'čki'), ('čkoj', 'čki'), ('čku', 'čki')]

add_adjective_inflection_forms(inflection_suffixes)

In [261]:
print_lexicon()

izmeniti [VERB]
uskladiti [VERB]
napraviti [VERB]
prenositi [VERB]
pridružiti [VERB]
posvetiti [VERB]
rušiti [VERB]
snositi [VERB]
nuditi [VERB]
odobriti [VERB]
opslužiti [VERB]
razotkriti [VERB]
oprostiti [VERB]
predložiti [VERB]
izgubiti [VERB]
odgovoriti [VERB]
zaključiti [VERB]
otvoriti [VERB]
suočiti [VERB]
težiti [VERB]
rijaliti [VERB]
izvršiti [VERB]
omogućiti [VERB]
pružiti [VERB]
naslediti [VERB]
razvodniti [VERB]
uhvatiti [VERB]
sprovoditi [VERB]
voditi [VERB]
ograničiti [VERB]
posetiti [VERB]
slediti [VERB]
protiviti [VERB]
odrediti [VERB]
uspostaviti [VERB]
stvoriti [VERB]
smanjiti [VERB]
otkloniti [VERB]
ostvariti [VERB]
protumačiti [VERB]
završiti [VERB]
napustiti [VERB]
izgraditi [VERB]
preporučiti [VERB]
zameniti [VERB]
nastupiti [VERB]
uložiti [VERB]
pratiti [VERB]
namestiti [VERB]
odvratiti [VERB]
učiniti [VERB]
povrediti [VERB]
nadoknaditi [VERB]
povratiti [VERB]
unaprediti [VERB]
osetiti [VERB]
ustanoviti [VERB]
razdvojiti [VERB]
umoriti [VERB]
usuditi [VERB]
arvani

### 2.3.3 suffix *-ški* and its inflections


Singular

| case | masculine | feminine | neutral | 
|------|-----------|----------|---------|
|nominative|češki|češka|češko|
|genitive|češkog|češke|češkog|
|dative|češkom|češkoj|češkom|
|acusative|češki|češku|češko|
|vocative|češki|češka|češko|
|instrumental|češkim|češkom|češkim|
|locative|češkom|češkoj|češkom|

Plural

| case | masculine | feminine | neutral | 
|------|-----------|----------|---------|
|nominative|češki|češke|češka|
|genitive|čeških|čeških|češke|
|dative|češkim|češkim|češkoj|
|acusative|češke|češke|češku|
|vocative|češki|češke|češka|
|instrumental|češkim|češkim|češkom|
|locative|češkim|češkim|češkoj|

Possible suffixes: *-ški*, *-škog*, *-škom*, *-ška*, *-ških*, *-škim*, *-ško*, *-ške*, *-škoj*, *-šku*

In [262]:
possessive_adjectives = list(set([w for w in ud_content if w.endswith('ški')]))

In [263]:
possessive_adjectives

['malteški',
 'strateški',
 'češki',
 'zoološki',
 'ideološki',
 'tehnološki',
 'muški',
 'njujorški',
 'bombaški',
 'norveški',
 'haški',
 'luksemburški']

In [264]:
for possesive_adj in possessive_adjectives:
    add_to_lexicon(possesive_adj, "ADJ")

In [265]:
inflection_suffixes = [('škog', 'ški'), ('škom', 'ški'), ('ška', 'ški'), ('ških', 'ški'), ('škim', 'ški'), ('ško', 'ški'), 
                      ('ške', 'ški'), ('škoj', 'ški'), ('šku', 'ški')]

add_adjective_inflection_forms(inflection_suffixes)

In [266]:
print_lexicon()

izmeniti [VERB]
uskladiti [VERB]
napraviti [VERB]
prenositi [VERB]
pridružiti [VERB]
posvetiti [VERB]
rušiti [VERB]
snositi [VERB]
nuditi [VERB]
odobriti [VERB]
opslužiti [VERB]
razotkriti [VERB]
oprostiti [VERB]
predložiti [VERB]
izgubiti [VERB]
odgovoriti [VERB]
zaključiti [VERB]
otvoriti [VERB]
suočiti [VERB]
težiti [VERB]
rijaliti [VERB]
izvršiti [VERB]
omogućiti [VERB]
pružiti [VERB]
naslediti [VERB]
razvodniti [VERB]
uhvatiti [VERB]
sprovoditi [VERB]
voditi [VERB]
ograničiti [VERB]
posetiti [VERB]
slediti [VERB]
protiviti [VERB]
odrediti [VERB]
uspostaviti [VERB]
stvoriti [VERB]
smanjiti [VERB]
otkloniti [VERB]
ostvariti [VERB]
protumačiti [VERB]
završiti [VERB]
napustiti [VERB]
izgraditi [VERB]
preporučiti [VERB]
zameniti [VERB]
nastupiti [VERB]
uložiti [VERB]
pratiti [VERB]
namestiti [VERB]
odvratiti [VERB]
učiniti [VERB]
povrediti [VERB]
nadoknaditi [VERB]
povratiti [VERB]
unaprediti [VERB]
osetiti [VERB]
ustanoviti [VERB]
razdvojiti [VERB]
umoriti [VERB]
usuditi [VERB]
arvani

In [267]:
data = pd.Series(ud_content)
data_freqs = data.value_counts()
data_freqs[:20]

rekao         332
godine        279
eu            261
takođe        166
bih           151
kada          141
kaže          132
zemlje        131
kosova        128
međutim       126
samo          120
još           117
evra          117
srbije        110
izjavio       107
trebalo       107
predsednik    106
odsto          99
više           98
ministar       94
dtype: int64